In [27]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "../../data/dynamic_supply_chain_logistics_dataset.csv"
df = pd.read_csv(file_path)

df.head()

,timestamp,vehicle_gps_latitude,vehicle_gps_longitude,fuel_consumption_rate,eta_variation_hours,traffic_congestion_level,warehouse_inventory_level,loading_unloading_time,handling_equipment_availability,order_fulfillment_status,...,iot_temperature,cargo_condition_status,route_risk_level,customs_clearance_time,driver_behavior_score,fatigue_monitoring_score,disruption_likelihood_score,delay_probability,risk_classification,delivery_time_deviation
0,2021-01-01 00:00:00,40.375568,-77.014318,5.136512,4.998009,5.927586,985.716862,4.951392,0.481294,0.761166,...,0.574400,0.777263,1.182116,0.502006,0.033843,0.978599,0.506152,0.885291,Moderate Risk,9.110682
1,2021-01-01 01:00:00,33.507818,-117.036902,5.101512,0.984929,1.591992,396.700206,1.030379,0.620780,0.196594,...,-9.753493,0.091839,9.611988,0.966774,0.201725,0.918586,0.980784,0.544178,High Risk,8.175281
2,2021-01-01 02:00:00,30.020640,-75.269224,5.090803,4.972665,8.787765,832.408935,4.220229,0.810933,0.152742,...,-6.491034,0.253529,6.570431,0.945627,0.264045,0.394215,0.998633,0.803322,High Risk,1.283594
3,2021-01-01 03:00:00,36.649223,-70.190529,8.219558,3.095064,0.045257,0.573283,0.530186,0.008525,0.811885,...,-0.151276,0.877576,0.548952,4.674035,0.362885,0.905444,0.993320,0.025977,High Risk,9.304897
4,2021-01-01 04:00:00,30.001279,-70.012195,5.000075,3.216077,8.004851,914.925067,3.620890,0.020083,0.053659,...,2.429448,0.262081,8.861443,3.445429,0.016957,0.258702,0.912433,0.991122,High Risk,7.752484


In [28]:
# @title 1st option - build from scratch
class LinearRegressionScratch:
    """
    Custom implementation of linear regression using gradient descent.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.coef_ = None
        self.intercept_ = None
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def fit(self, X, y):
        """
        Fits the linear regression model to the given data using gradient descent.
        Args:
            X: A numpy array of shape (n_samples, n_features) representing the input data.
            y: A numpy array of shape (n_samples,) representing the target values.
        """
        # Initialize coefficients (slope) and intercept to zero or random small values
        self.coef_ = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))
        self.intercept_ = np.mean(y - np.dot(X, self.coef_))

        m = len(y)  # number of training examples

        for _ in range(self.n_iterations):
            # Calculate the predictions
            y_pred = self.predict(X)

            # Compute the residuals (errors)
            error = (y_pred - y)

            # Calculate the gradient for intercept (slope) and coefficients (slope)
            intercept_gradient = (1/m) * np.sum(error)
            coef_gradient = (1/m) * np.dot(X.T, error)

            # Update the parameters using the gradients
            self.intercept_ -= self.learning_rate * intercept_gradient
            self.coef_ -= self.learning_rate * coef_gradient

    def predict(self, X):
        """
        Predicts the target values for new data.
        Args:
            X: A numpy array of shape (n_samples, n_features) representing the new input data.
        Returns:
            A numpy array of shape (n_samples,) representing the predicted target values.
        """
        return np.dot(X, self.coef_) + self.intercept_

In [29]:
# Cleaning Data
numerical_list = [x for x in df.columns if df[x].dtype in ('int64','float64')]
print(numerical_list)

['vehicle_gps_latitude', 'vehicle_gps_longitude', 'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability', 'order_fulfillment_status', 'weather_condition_severity', 'port_congestion_level', 'shipping_costs', 'supplier_reliability_score', 'lead_time_days', 'historical_demand', 'iot_temperature', 'cargo_condition_status', 'route_risk_level', 'customs_clearance_time', 'driver_behavior_score', 'fatigue_monitoring_score', 'disruption_likelihood_score', 'delay_probability', 'delivery_time_deviation']


In [30]:
# Cleaning Data
from sklearn.preprocessing import MinMaxScaler

df_cleaned = df.drop(columns=['risk_classification'])
df_cleaned = df_cleaned.drop(columns=['timestamp'])

# Handle missing values (remove rows with missing data)
df_cleaned = df_cleaned.dropna()

# Detect and remove outliers using the IQR method
for i in numerical_list:
    Q1 = df_cleaned[i].quantile(0.25)
    Q3 = df_cleaned[i].quantile(0.75)
    IQR = Q3 - Q1
    df_cleaned = df_cleaned[df_cleaned[i] <= (Q3+(1.5*IQR))]
    df_cleaned = df_cleaned[df_cleaned[i] >= (Q1-(1.5*IQR))]
    df_cleaned = df_cleaned.reset_index(drop=True)
# Define acceptable range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Normalize numerical features using Min-Max Scaling
scaler = MinMaxScaler()
df_cleaned[df_cleaned.columns] = scaler.fit_transform(df_cleaned[df_cleaned.columns])

df_cleaned.head()

,vehicle_gps_latitude,vehicle_gps_longitude,fuel_consumption_rate,eta_variation_hours,traffic_congestion_level,warehouse_inventory_level,loading_unloading_time,handling_equipment_availability,order_fulfillment_status,weather_condition_severity,...,historical_demand,iot_temperature,cargo_condition_status,route_risk_level,customs_clearance_time,driver_behavior_score,fatigue_monitoring_score,disruption_likelihood_score,delay_probability,delivery_time_deviation
0,0.518778,0.859714,0.011725,0.999716,0.592759,0.985718,0.989198,0.481294,0.761166,0.359066,...,0.000078,0.265758,0.777263,0.118207,0.000446,0.033843,0.978599,0.351908,0.885291,0.925890
1,0.175391,0.059262,0.008719,0.426418,0.159199,0.396700,0.117862,0.620781,0.196593,0.230660,...,0.526640,0.006195,0.091839,0.961199,0.103728,0.201725,0.918586,0.974782,0.544176,0.847940
2,0.001032,0.894616,0.007799,0.996095,0.878777,0.832410,0.826718,0.810935,0.152740,0.027210,...,0.151015,0.088188,0.253529,0.657041,0.099028,0.264045,0.394215,0.998206,0.803321,0.273633
3,0.332461,0.996189,0.276521,0.727866,0.004526,0.000573,0.006708,0.008525,0.811885,0.616619,...,0.245498,0.247520,0.877576,0.054891,0.927563,0.362885,0.905444,0.991234,0.025974,0.942075
4,0.000064,0.999756,0.000006,0.745154,0.800485,0.914926,0.693531,0.020083,0.053658,0.952385,...,0.206686,0.312379,0.262081,0.886144,0.654540,0.016957,0.258702,0.885083,0.991122,0.812707


In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

# Prepare Data
features = ['fuel_consumption_rate', 'delay_probability', 'traffic_congestion_level', 'loading_unloading_time', 'handling_equipment_availability']
X = df_cleaned[features].to_numpy()
y = df_cleaned['delivery_time_deviation'].to_numpy()

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


class PolyRegressionScratch:
    """
    Custom implementation of polynomial regression using gradient descent.
    Supports arbitrary polynomial degrees.
    """
    def __init__(self, degree=2, alpha=0.0001, iterations=1000):
        self.degree = degree
        self.alpha = alpha
        self.iterations = iterations
        self.theta = None  # Model coefficients

    def transform_features(self, X):
        """ Apply polynomial transformation to X """
        poly = PolynomialFeatures(degree=self.degree, include_bias=False)
        return poly.fit_transform(X)

    def predict(self, X):
        """ Compute predictions using the trained model """
        return np.dot(X, self.theta)

    def compute_cost(self, X, y):
        """ Compute Mean Squared Error (MSE) loss """
        m = len(y)
        predictions = self.predict(X)
        return (1 / (2 * m)) * np.sum((predictions - y) ** 2)

    def fit(self, X, y):
        """ Train the model using gradient descent """
        X_poly = self.transform_features(X)  # Transform features to polynomial terms
        m, n = X_poly.shape  # Number of samples, number of features
        self.theta = np.zeros(n)  # Initialize parameters

        # Gradient Descent
        for i in range(self.iterations):
            predictions = self.predict(X_poly)
            gradients = (1 / m) * np.dot(X_poly.T, (predictions - y))  # Compute gradients
            self.theta -= self.alpha * gradients  # Update parameters

            # Print cost every 100 iterations
            if i % 100 == 0:
                print(f"Iteration {i}, Cost: {self.compute_cost(X_poly, y):.4f}")

        print("Final Parameters:", self.theta)


# Instantiate Model with Degree = 2
poly_reg_scratch = PolyRegressionScratch(degree=2, alpha=0.0001, iterations=1000)

# Train Model
poly_reg_scratch.fit(X_train, y_train)

# Make Predictions
X_test_poly = poly_reg_scratch.transform_features(X_test)  # Transform test data
y_pred = poly_reg_scratch.predict(X_test_poly)

# Evaluation Metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print Evaluation Results
print("\nModel Evaluation:")
print(f"R-squared (R²): {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Iteration 0, Cost: 0.2382
Iteration 100, Cost: 0.2313
Iteration 200, Cost: 0.2247
Iteration 300, Cost: 0.2183
Iteration 400, Cost: 0.2123
Iteration 500, Cost: 0.2066
Iteration 600, Cost: 0.2011
Iteration 700, Cost: 0.1958
Iteration 800, Cost: 0.1908
Iteration 900, Cost: 0.1861
Final Parameters: [0.00988495 0.03742773 0.02648652 0.02129507 0.01626911 0.00555358
 0.0068171  0.0048961  0.00392434 0.00298371 0.03167086 0.01821971
 0.01467531 0.01119112 0.01969725 0.01030286 0.00782877 0.0147602
 0.0064491  0.01060944]

Model Evaluation:
R-squared (R²): -2.0318
Root Mean Squared Error (RMSE): 0.6054
